<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-01-30 11:42:46
-------------------
qualified stocks: 86
with latest results: 80
still star stocks: 44


-------------------
Initial Investment:  1.04 C
CY Investment:  1.17 C
Reserve:  2.39 L
Current:  1.14 C
-------------------
Today PnL: 68.89 K (0.61%)
Current PnL: -16.92 L (-14.42%)
CY Booked + Current PnL: -3.65 L (-3.11%)
-------------------
Total profit:  2.37 L
Total loss:  -19.29 L
-------------------
Total Booked + Current PnL: 9.46 L (9.07%)
Total Booked PnL: 26.38 L (25.3%)
Curr Year Booked PnL: 13.27 L (11.65%)
Prev Year Booked PnL: 13.11 L (12.57%)
Est FTT:  1.85 C
Est FTT PnL: 70.95 L (62.31%)
-------------------
Est LTT:  2.44 C
Est LTT PnL: 1.30 C (114.23%)
Deployed:  1.04 C
Current:  1.14 C
CAGR/XIRR %: 7.19%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HINDUNILVR,2430.78,2723.00,-13.28,X-LC,79.07,201600.0,-2586.0,27135.0,0.67,-1.27,13.46,12.02,-3.0,-0.10,1.81,11.29,XY25,NTT,FMCG
65,SBILIFE,1483.67,1928.65,-9.37,H-LC,89.53,99736.0,-1154.0,31417.0,0.40,-1.14,31.50,29.99,NaN,-0.04,0.89,9.55,AR,ATH,INSURANCE
69,SIS,477.00,528.00,57.98,M-SC,36.05,56925.0,-24165.0,32834.0,0.87,-29.80,57.68,10.69,-28.0,-0.74,0.51,9.73,X40N,NTT,MISC
31,HAVELLS,1588.50,2077.12,-2.01,X-LC,44.77,97414.0,-2662.0,33442.0,1.18,-2.66,34.33,30.76,NaN,-0.08,0.87,20.43,X40,ATH,ELECTRICAL
9,AXISBANK,986.10,1317.30,-31.10,H-LC,32.56,100429.0,-153.0,33935.0,0.12,-0.15,33.79,33.59,NaN,-0.00,0.90,3.85,X40,ATH,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TITAGARH,1117.46,1548.00,-10.18,M-SC,25.58,161810.0,-20336.0,90517.0,4.83,-11.16,55.94,38.53,-40.0,-0.22,1.45,20.56,XY24,NTT,ENGINEERING
80,UJJIVANSFB,59.50,60.00,-35.16,M-SC,91.86,59993.0,-41157.0,42007.0,3.89,-40.69,70.02,0.84,-27.0,-0.98,0.54,10.35,X40N,NTT,BANKS
35,HONAUT,42646.15,58518.52,-14.38,X-MC,72.09,124449.0,-3489.0,51111.0,3.60,-2.73,41.07,37.22,97.0,-0.07,1.12,14.04,X40N,ATH,ELECTRICAL
36,INDIGOPNTS,1407.73,2061.00,-23.18,H-SC,17.44,155000.0,-19559.0,100564.0,3.22,-11.20,64.88,46.41,-1.0,-0.19,1.39,4.58,X40N,NTT,PAINTS
55,RBLBANK,230.61,388.00,-40.24,M-SC,84.88,165242.0,-69058.0,228959.0,3.01,-29.47,138.56,68.25,-48.0,-0.30,1.48,8.82,XY24,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,798.82,1065.0,-59.73,M-LC,54.65,144320.0,-19438.0,74007.0,-6.45,-11.87,51.28,33.32,-49.0,-0.26,1.29,0.00,XY24,NTT,AUTO
61,SAMMAANCAP,170.35,326.0,-173.72,M-SC,51.16,85692.0,-16518.0,109909.0,-2.14,-16.16,128.26,91.37,NaN,-0.15,0.77,7.26,XR,NTT,FINANCE
3,ACC,2579.05,3906.0,-21.61,M-MC,52.33,143412.0,-44859.0,141720.0,-1.87,-23.83,98.82,51.45,-37.0,-0.32,1.29,5.65,XY24,BTT,CEMENT
44,LAOPALA,369.40,464.0,-30.46,M-SC,6.98,71635.0,-29211.0,55037.0,-1.78,-28.97,76.83,25.61,-50.0,-0.53,0.64,0.00,AR,NTT,CERAMICS
38,INFY,1436.24,2275.0,3.74,X-LC,61.63,193242.0,43873.0,43364.0,-1.23,29.37,22.44,58.40,33.0,1.01,1.73,34.84,X40,BTT,IT


In [12]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,308.25,420.0,33.17,M-LC,88.37,102646.0,924.0,35957.0,-0.45,0.91,35.03,36.25,33.0,0.03,0.92,45.20,XR,NTT,IT
46,LUXIND,1448.19,2442.0,18.05,M-SC,9.30,148827.0,11249.0,83165.0,-1.17,8.18,55.88,68.62,6.0,0.14,1.34,45.95,X40N,NTT,TEXTILES
27,GICRE,335.06,583.0,3.05,H-MC,30.23,178362.0,31941.0,76410.0,0.79,21.81,42.84,74.00,109.0,0.42,1.60,31.92,XY24,BTT,INSURANCE
63,SBICARD,765.91,1075.0,-26.44,H-MC,98.84,180997.0,1008.0,71621.0,1.16,0.56,39.57,40.36,-1.0,0.01,1.62,16.02,XY24,NTT,FINANCE
64,SBIETFPB,241.24,400.0,NaN,NaN,63.95,186578.0,3959.0,116219.0,1.28,2.17,62.29,65.81,0.0,0.03,1.67,8.09,XY24,BTT,BANKS


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,308.25,420.0,33.17,M-LC,88.37,102646.0,924.0,35957.0,-0.45,0.91,35.03,36.25,33.0,0.03,0.92,45.20,XR,NTT,IT
13,BANKINDIA,112.90,190.0,-24.59,M-MC,95.35,98666.0,-1589.0,70053.0,0.26,-1.59,71.00,68.29,-12.0,-0.02,0.89,22.98,XR,NTT,BANKS
1,AARTIIND,487.04,919.0,-0.81,M-SC,94.19,72787.0,-8549.0,80684.0,1.86,-10.51,110.85,88.69,-8.0,-0.11,0.65,9.55,XR,NTT,CHEMICALS
70,SONACOMS,578.05,1006.0,-32.15,M-MC,10.47,88366.0,-12793.0,87686.0,0.54,-12.65,99.23,74.03,-8.0,-0.15,0.79,1.56,AR,BTT,AUTO
61,SAMMAANCAP,170.35,326.0,-173.72,M-SC,51.16,85692.0,-16518.0,109909.0,-2.14,-16.16,128.26,91.37,NaN,-0.15,0.77,7.26,XR,NTT,FINANCE


In [14]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,308.25,420.0,33.17,M-LC,88.37,102646.0,924.0,35957.0,-0.45,0.91,35.03,36.25,33.0,0.03,0.92,45.20,XR,NTT,IT
10,BAJAJFINSV,1473.47,2392.0,-10.92,X-LC,96.51,139651.0,24720.0,46923.0,1.21,21.51,33.60,62.34,11.0,0.53,1.25,21.27,X40,BTT,FINANCE
27,GICRE,335.06,583.0,3.05,H-MC,30.23,178362.0,31941.0,76410.0,0.79,21.81,42.84,74.00,109.0,0.42,1.60,31.92,XY24,BTT,INSURANCE
63,SBICARD,765.91,1075.0,-26.44,H-MC,98.84,180997.0,1008.0,71621.0,1.16,0.56,39.57,40.36,-1.0,0.01,1.62,16.02,XY24,NTT,FINANCE
38,INFY,1436.24,2275.0,3.74,X-LC,61.63,193242.0,43873.0,43364.0,-1.23,29.37,22.44,58.40,33.0,1.01,1.73,34.84,X40,BTT,IT


In [15]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BAJFINANCE,7055.29,9150.0,-21.13,X-LC,100.00,247208.0,28494.0,36438.0,2.77,13.03,14.74,29.69,19.0,0.78,2.22,26.99,X40,BTT,FINANCE
63,SBICARD,765.91,1075.0,-26.44,H-MC,98.84,180997.0,1008.0,71621.0,1.16,0.56,39.57,40.36,-1.0,0.01,1.62,16.02,XY24,NTT,FINANCE
20,COFFEEDAY,59.14,80.0,-127.43,L-SC,97.67,49824.0,-63725.0,103778.0,0.00,-56.12,208.29,35.27,NaN,-0.61,0.45,14.72,XR,NTT,HOTELS
10,BAJAJFINSV,1473.47,2392.0,-10.92,X-LC,96.51,139651.0,24720.0,46923.0,1.21,21.51,33.60,62.34,11.0,0.53,1.25,21.27,X40,BTT,FINANCE
13,BANKINDIA,112.90,190.0,-24.59,M-MC,95.35,98666.0,-1589.0,70053.0,0.26,-1.59,71.00,68.29,-12.0,-0.02,0.89,22.98,XR,NTT,BANKS


In [16]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.25
1,25,43.14
2,50,73.65


In [17]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    40.86
X40     18.11
X40N    11.56
AR      10.86
XR      10.25
XY25     5.37
X200     1.58
SR       1.38
Name: CurrAlloc%, dtype: float64

In [18]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-LC    24.70
M-SC    23.52
H-SC    15.78
M-MC     8.00
X-MC     6.33
H-MC     6.15
X-SC     4.65
H-LC     3.28
M-LC     3.27
L-SC     2.62
Name: CurrAlloc%, dtype: float64

In [19]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FINANCE,12.93,-13.35,61.07
BANKS,12.64,-17.18,75.59
FMCG,10.44,-4.38,36.02
IT,9.73,-11.98,66.55
PAINTS,6.00,-22.91,59.28
MISC,5.69,-19.44,57.32
INSURANCE,5.29,0.95,48.40
CHEMICALS,4.17,-16.54,93.34
AUTO,3.14,-17.33,68.92


In [20]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3099805.0,30
XR,1087206.0,14
X40N,982311.0,14
AR,871501.0,10
X40,698748.0,12
SR,191618.0,2
XY25,121881.0,3
X200,42184.0,1


In [21]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
M-SC,2034274.0,24
H-SC,1779315.0,14
X-LC,801576.0,15
M-MC,712861.0,7
X-SC,384399.0,5
H-MC,369144.0,5
L-SC,305933.0,4
X-MC,274174.0,5
M-LC,182675.0,3


In [22]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
M-SC       XY24      1008099.0     10
H-SC       XY24       752968.0      5
M-MC       XY24       555122.0      5
X-LC       X40        474249.0      8
H-SC       X40N       430246.0      5
M-SC       XR         383407.0      5
           X40N       367710.0      5
H-MC       XY24       321115.0      4
H-SC       AR         317695.0      1
           XR         278406.0      3
L-SC       XR         169015.0      2
X-LC       AR         163262.0      3
M-LC       XY24       146718.0      2
M-SC       AR         138260.0      2
X-SC       SR         122876.0      1
X-LC       XY25       121881.0      3
H-LC       X40        103267.0      2
X-SC       XR         102339.0      1
           X40N        93121.0      2
X-MC       X40N        91234.0      2
M-MC       AR          87686.0      1
L-SC       XY24        79208.0      1
X-MC       AR          75471.0      1
M-MC       XR          70053.0      1
M-SC       SR          68742.0      1
           X40         68056.0      1
X-SC       XY24        66063.0      1
L-SC       AR          57710.0      1
X-MC       XY24        54293.0      1
           X40         53176.0      1
H-MC       XR          48029.0      1
X-LC       X200        42184.0      1
M-LC       XR          35957.0      1
H-LC       AR          31417.0      1

In [23]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
